In [1]:
import pandas as pd
import requests
from tqdm import tqdm_notebook
import json
from api_keys import api_key
import os
from sqlalchemy import create_engine
import numpy as np

Import csv of imdb_id's

In [2]:
imdb_data = "../unique_indmID.csv"
imdb_id = pd.read_csv(imdb_data,skiprows=[1], low_memory=False)

View data

In [3]:
imdb_id = imdb_id['imdb_id']

Create functions to parse data as API calls are made.  By doing this it reduces the need for itterows or additional for loops later in the code.

In [4]:
def writer_list(writer, imdb):
    final_list = []
    name_list = writer.split(',')
    for index in name_list:
        name = index.split(' (')[0]
        final_list.append({'imdb_id' : imdb,
                         'first_name' : name.rsplit(' ', 1)[0],
                         'last_name' : name.split()[-1]})
    return final_list

In [5]:
def process_list(name_string, imdb):
    final_list = []
    name_list = name_string.split(',')
    for index in name_list:
        final_list.append({'imdb_id' : imdb,
                               'first_name' : index.rsplit(' ', 1)[0],
                               'last_name' : index.split()[-1]})
    return final_list

API calls and parsing by use of functions

In [6]:
url = "http://www.omdbapi.com//?i="
api = f"&apikey={api_key}"
actor_data = []
director_data = []
writer_data = []

for id_ in tqdm_notebook(imdb_id):
    try:
        response = requests.get(url + id_+api).json()
    except JSONDecodeError:
        print(f'error at {id_}')
        response = requests.get(url + id_+api).json()
    try:
        director = response['Director']
        temp_list = process_list(director, id_)
        for item in temp_list:
             director_data.append(item)
        actor = response['Actors']
        temp_list = process_list(actor, id_)
        for item in temp_list:
             actor_data.append(item)
        writer = response['Writer']
        temp_list = writer_list(writer, id_)
        for item in temp_list:
             writer_data.append(item)
    except KeyError:
        print(f'KeyError at {id_}')
        director_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})
        actor_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})
        writer_data.append({'imdb_id' : id_,
                         'first_name' : 'No Data',
                         'last_name' : 'No Data'})

KeyError at tt1628842
KeyError at tt2697688



Convert list of dictionaries to DataFrames

In [ ]:
ratings_data = []
awards_data = []

for id_ in tqdm_notebook(imdb_id):
    try:
        response = requests.get(url + id_+api).json()
    except JSONDecodeError:
        print(f'error at {id_}')
        response = requests.get(url + id_+api).json()
    try:
        rating = response['Rated']
        ratings_data.append({'imdb_id' : id_,
                             'ratings' : rating})
        award = response['Awards']
        awards_data.append({'imdb_id' : id_,
                            'awards' : award})
    except KeyError:
        print(f'KeyError at {id_}')
        awards_data.append({'imdb_id' : id_,
                            'awards' : 'No Data'})
        ratings_data.append({'imdb_id' : id_,
                             'ratings' : 'No Data'})
    

In [7]:
#actor_all_df = pd.DataFrame(actor_data)
#writer_all_df = pd.DataFrame(writer_data)
#director_all_df = pd.DataFrame(director_data)
ratings_all_df = pd.DataFrame(ratings_data)
awards_all_df = pd.DataFrame(awards_data)

Replace blanks left by individuals with only one name.

In [8]:
actor_all_df.replace('', 'N/A', inplace = True)
writer_all_df.replace('', 'N/A', inplace = True)
director_all_df.replace('', 'N/A', inplace = True)

Create individual DataFrames which will then be pushed to the database as tables.

In [29]:
film_actor_id_df = actor_all_df
actor_id = film_actor_id_df.groupby(['last_name', 'first_name']).ngroup()
film_actor_id_df['actor_id'] = actor_id
film_actor_id_df['actor_id'] = 'a' + film_actor_id_df['actor_id'].astype(str)
film_actor_df = film_actor_id_df.drop(['first_name', 'last_name'], axis=1)
film_actor_df.drop_duplicates(subset=['imdb_id', 'actor_id'], inplace = True)
actor_df = film_actor_id_df.drop(['imdb_id'], axis=1)
actor_df.drop_duplicates(subset='actor_id', inplace = True)

In [24]:
film_writer_id_df = writer_all_df
writer_id = film_writer_id_df.groupby(['last_name', 'first_name']).ngroup()
film_writer_id_df['writer_id'] = writer_id
film_writer_id_df['writer_id'] = 'w' + film_writer_id_df['writer_id'].astype(str)
film_writer_df = film_writer_id_df.drop(['first_name', 'last_name'], axis=1)
film_writer_df.drop_duplicates(subset=['imdb_id', 'writer_id'], inplace = True)
writer_df = film_writer_id_df.drop(['imdb_id'], axis=1)
writer_df.drop_duplicates(subset='writer_id', inplace = True)

In [32]:
film_director_id_df = director_all_df
director_id = film_director_id_df.groupby(['last_name', 'first_name']).ngroup()
film_director_id_df['director_id'] = director_id
film_director_id_df['director_id'] = 'd' + film_director_id_df['director_id'].astype(str)
film_director_df = film_director_id_df.drop(['first_name', 'last_name'], axis=1)
film_director_df.drop_duplicates(subset=['imdb_id', 'director_id'], inplace = True)
director_df = film_director_id_df.drop(['imdb_id'], axis=1)
director_df.drop_duplicates(subset='director_id', inplace = True)

Confirm table structure and headers

In [39]:
film_actor_df.head(1)

,imdb_id,actor_id
0,tt0113497,a81066


In [40]:
actor_df.head(1)

,first_name,last_name,actor_id
0,Robin,Williams,a81066


In [42]:
film_writer_df.head(1)

,imdb_id,writer_id
0,tt0113497,w20063


In [43]:
writer_df.head(1)

,first_name,last_name,writer_id
0,Jonathan,Hensleigh,w20063


In [44]:
film_director_df.head(1)

,imdb_id,director_id
0,tt0113497,d9185


In [45]:
director_df.head(1)

,first_name,last_name,director_id
0,Joe,Johnston,d9185


Initiate connection to database and verify tables present in database

In [37]:
rds_connection_string = "postgres:password@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['movie',
 'film_actor',
 'actor',
 'spoken_language',
 'film_spoken_language',
 'genre',
 'film_genre',
 'production_company',
 'film_production_company',
 'writer',
 'film_writer',
 'director',
 'film_director',
 'country',
 'film_country',
 'collection',
 'w_collec']

Push DataFrames to the database as tables

In [38]:
actor_df.to_sql(name='actor', con=engine, if_exists='append', index=False)
director_df.to_sql(name='director', con=engine, if_exists='append', index=False)
writer_df.to_sql(name='writer', con=engine, if_exists='append', index=False)
film_actor_df.to_sql(name='film_actor', con=engine, if_exists='append', index=False)
film_director_df.to_sql(name='film_director', con=engine, if_exists='append', index=False)
film_writer_df.to_sql(name='film_writer', con=engine, if_exists='append', index=False)

Create csv files for later use if needed.

In [47]:
actor_df.to_csv('actor_df.csv', index=False)
director_df.to_csv('director_df.csv', index=False)
writer_df.to_csv('writer_df.csv', index=False)
film_actor_df.to_csv('film_actor_df.csv', index=False)
film_director_df.to_csv('film_director_df.csv', index=False)
film_writer_df.to_csv('film_writer_df.csv', index=False)